<a href="https://colab.research.google.com/github/phyllsmoyo/Coursera_Capstone/blob/main/Capstone_Project_Segmenting_and_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Notebook to explore and cluster the neighborhoods in Toronto.

**1. Web Scrapping**


Import the required Libraries.


In [1]:
import pandas as pd #data analysis and manipulation tool
import numpy as np #numerical computation tool
import matplotlib.pyplot as plt #python plotting library


Now we assign the website link through which we are going to scrape the data and assign it to variable named url.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Pandas ```read_html``` is pretty simple and works well on many Wikipedia pages since the tables are not complicated. 


In [3]:
table1 = pd.read_html(url, match='Borough') #Use the match parameter in order to return the specigic table
print(f'Total tables in the url: {len(table1)}')

Total tables in the url: 1


In [4]:
df = table1[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


1. The dataframe should consist of three columns: 
    - PostalCode, 
    - Borough, and 
    - Neighborhood

2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    - More than one neighborhood can exist in one postal code area. 
        - For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
        - These two rows will be combined into one row with the neighborhoods separated with a comma.
    - If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

3. Clean the Notebook and add Markdown cells to explain the work and any assumptions made.

In [6]:
#Creat a filter for to remove the Boroughs not assigned.
filter = df['Borough'] == 'Not assigned'

#Filter out the "Not assigned" Boroughs 
df_BA = df[~filter]
df_BA.head()


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In the last cell of the notebook, use the .shape method to print the number of rows in the dataframe.

In [9]:
print(f'The dataframe has {df_BA.shape[0]} rows')


The dataframe has 103 rows
